In [ ]:
#SCALED DOT PRODUCT
from IPython import get_ipython
from IPython.display import display
from tensorflow import matmul, cast, float32, math
from tensorflow.math import sqrt
from tensorflow.keras.layers import Layer
from tensorflow.keras.activations import softmax
import numpy as np

class DotProductAttention(Layer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def call(self, queries, keys, values, *, d_k, mask=None):
    scores = matmul(queries, keys, transpose_b=True) / sqrt(cast(d_k, float32))
    if mask is not None:
      scores += -1e9 * mask
    weights = softmax(scores)
    return matmul(weights, values)

batch_size = 32
input_seq_length = 10
d_k = 64
d_v = 64

random = np.random.default_rng(seed=42)
queries = random.random((batch_size, input_seq_length, d_k))
keys = random.random((batch_size, input_seq_length, d_k))
values = random.random((batch_size, input_seq_length, d_v))
attention = DotProductAttention()
print(attention(queries, keys, values, d_k=d_k))

In [ ]:
from IPython import get_ipython
from IPython.display import display
# %%
from tensorflow import math, matmul, reshape, shape, transpose, cast, float32, concat
from tensorflow.keras.layers import Dense, Layer
from tensorflow.keras.backend import softmax
# Implementing the Scaled-Dot Product Attention

class DotProductAttention(Layer):
  def __init__(self, **kwargs):
    super().__init__(**kwargs)

  def call(self, queries, keys, values, mask=None):
    d_k = queries.shape[-1]
    # Scoring the queries against the keys after transposing the latter, and scaling
    scores = matmul(queries, keys, transpose_b=True) / math.sqrt(cast(d_k, float32))
    # Apply mask to the attention scores
    if mask is not None:
      scores += -1e9 * mask
    # Computing the weights by a softmax operation
    weights = softmax(scores)
    # Computing the attention by a weighted sum of the value vectors
    return matmul(weights, values)
    # Implementing the Multi-Head Attention

class MultiHeadAttention(Layer):
  def __init__(self, h, d_k, d_v, d_model, **kwargs):
    super().__init__(**kwargs)
    self.attention = DotProductAttention() # Scaled dot product attention
    self.heads = h # Number of attention heads to use
    self.d_k = d_k # Dimensionality of the linearly projected queries and keys
    self.d_v = d_v # Dimensionality of the linearly projected values
    self.d_model = d_model # Dimensionality of the model
    self.W_q = Dense(d_k) # Learned projection matrix for the queries
    self.W_k = Dense(d_k) # Learned projection matrix for the keys
    self.W_v = Dense(d_v) # Learned projection matrix for the values
    self.W_o = Dense(d_model) # Learned projection matrix for the multi-head output

  def reshape_tensor(self, x, heads, flag):
    if flag:
      # Tensor shape after reshaping and transposing:
      # (batch_size, heads, seq_length, -1)
      x = reshape(x, shape=(shape(x)[0], shape(x)[1], heads, -1))
      x = transpose(x, perm=(0, 2, 1, 3))
    else:
        x = transpose(x, perm=(0, 2, 1, 3))
        x_shape = shape(x)
        new_shape = (x_shape[0], x_shape[1], x_shape[2] * x_shape[3])
        x = reshape(x, new_shape)

    return x

  def call(self, queries, keys, values, mask=None):
    # Rearrange the queries to be able to compute all heads in parallel
    q_reshaped = self.reshape_tensor(self.W_q(queries), self.heads, True)
    # Resulting tensor shape: (batch_size, heads, input_seq_length, -1)
    # Rearrange the keys to be able to compute all heads in parallel
    k_reshaped = self.reshape_tensor(self.W_k(keys), self.heads, True)
    # Resulting tensor shape: (batch_size, heads, input_seq_length, -1)
    # Rearrange the values to be able to compute all heads in parallel
    v_reshaped = self.reshape_tensor(self.W_v(values), self.heads, True)
    # Resulting tensor shape: (batch_size, heads, input_seq_length, -1)
    # Compute the multi-head attention output using the reshaped queries,
    # keys, and values
    o_reshaped = self.attention(q_reshaped, k_reshaped, v_reshaped, mask=mask)
    # Resulting tensor shape: (batch_size, heads, input_seq_length, -1)
    # Rearrange back the output into concatenated form
    output = self.reshape_tensor(o_reshaped, self.heads, False)
    # Resulting tensor shape: (batch_size, input_seq_length, d_model)
    return self.W_o(output)
# %%
from numpy import random
input_seq_length = 5 # Maximum length of the input sequence
h = 8 # Number of self-attention heads
d_k = 64 # Dimensionality of the linearly projected queries and keys
d_v = 64 # Dimensionality of the linearly projected values
d_model = 512 # Dimensionality of the model sub-layers' outputs
batch_size = 64 # Batch size from the training process
queries = random.random((batch_size, input_seq_length, d_k))
keys = random.random((batch_size, input_seq_length, d_k))
values = random.random((batch_size, input_seq_length, d_v))
multihead_attention = MultiHeadAttention(h, d_k, d_v, d_model)
print(multihead_attention(queries, keys, values))

In [1]:
import tensorflow as tf

class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)  # Dense layer for the encoder hidden states
        self.W2 = tf.keras.layers.Dense(units)  # Dense layer for the decoder hidden state
        self.V = tf.keras.layers.Dense(1)       # Dense layer to compute alignment scores

    def call(self, query, values):
        """
        Args:
            query: Decoder hidden state (shape: [batch_size, hidden_size]).
            values: Encoder outputs (shape: [batch_size, seq_len, hidden_size]).
        Returns:
            context_vector: Weighted sum of encoder outputs (shape: [batch_size, hidden_size]).
            attention_weights: Attention weights (shape: [batch_size, seq_len]).
        """
        # Add time axis to query for broadcasting (shape: [batch_size, 1, hidden_size])
        query_with_time_axis = tf.expand_dims(query, 1)

        # Compute the alignment scores (shape: [batch_size, seq_len, 1])
        score = self.V(tf.nn.tanh(self.W1(values) + self.W2(query_with_time_axis)))

        # Remove the last axis (shape: [batch_size, seq_len])
        attention_weights = tf.nn.softmax(score, axis=1)

        # Compute the context vector as the weighted sum of values (shape: [batch_size, hidden_size])
        context_vector = tf.reduce_sum(attention_weights * values, axis=1)

        return context_vector, attention_weights

# Example usage
if __name__ == "__main__":
    # Define batch size, sequence length, and hidden size
    batch_size = 64
    seq_len = 10
    hidden_size = 256
    attention_units = 128

    # Instantiate the attention layer
    attention = BahdanauAttention(units=attention_units)

    # Simulated encoder outputs (values) and decoder hidden state (query)
    encoder_outputs = tf.random.normal([batch_size, seq_len, hidden_size])
    decoder_hidden_state = tf.random.normal([batch_size, hidden_size])

    # Apply the attention mechanism
    context_vector, attention_weights = attention(decoder_hidden_state, encoder_outputs)

    print("Context vector shape:", context_vector.shape)  # Expected: [batch_size, hidden_size]
    print("Attention weights shape:", attention_weights.shape)  # Expected: [batch_size, seq_len]


Context vector shape: (64, 256)
Attention weights shape: (64, 10, 1)


In [2]:
decoder_hidden_state

<tf.Tensor: shape=(64, 256), dtype=float32, numpy=
array([[-1.0162307 ,  0.2542984 , -1.1475248 , ...,  1.2792453 ,
         0.08146834,  1.110824  ],
       [ 0.09520556,  0.15147667,  0.30544576, ..., -0.96792173,
         0.29477042, -0.31000718],
       [ 0.1262379 ,  1.0118979 , -0.391723  , ..., -1.1618767 ,
        -0.38834393, -0.6393011 ],
       ...,
       [ 0.71681315,  0.47909588, -1.1250935 , ..., -2.7017155 ,
        -2.9282985 ,  0.15480897],
       [ 0.42129475, -0.25539428,  0.5809801 , ...,  2.3172004 ,
        -2.638371  , -1.4490504 ],
       [ 0.76044065,  0.5144072 , -0.9479262 , ...,  1.6367507 ,
         1.2844629 , -0.68204767]], dtype=float32)>

In [3]:
decoder_hidden_state.shape

TensorShape([64, 256])

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense

class LuongAttention(Layer):
    def __init__(self, attention_type, hidden_size):
        super(LuongAttention, self).__init__()
        self.attention_type = attention_type
        self.hidden_size = hidden_size

        if attention_type == "general":
            self.attention_weight = Dense(hidden_size)
        elif attention_type == "concat":
            self.attention_weight = Dense(hidden_size)
            self.v = tf.Variable(tf.random.normal([hidden_size]), trainable=True)

    def score(self, hidden, encoder_outputs):
        if self.attention_type == "dot":
            # Dot product between hidden state and encoder outputs
            return tf.matmul(encoder_outputs, tf.expand_dims(hidden, axis=-1))[:, :, 0]

        elif self.attention_type == "general":
            # Linear transformation followed by dot product
            energy = self.attention_weight(encoder_outputs)
            return tf.matmul(energy, tf.expand_dims(hidden, axis=-1))[:, :, 0]

        elif self.attention_type == "concat":
            # Concatenate hidden state with encoder outputs
            hidden_expanded = tf.expand_dims(hidden, axis=1)
            hidden_expanded = tf.tile(hidden_expanded, [1, tf.shape(encoder_outputs)[1], 1])
            concat_input = tf.concat([hidden_expanded, encoder_outputs], axis=-1)
            energy = tf.tanh(self.attention_weight(concat_input))
            return tf.reduce_sum(energy * self.v, axis=2)

        else:
            raise ValueError("Unknown attention type: {}".format(self.attention_type))

    def call(self, hidden, encoder_outputs):
        # Compute alignment scores
        alignment_scores = self.score(hidden, encoder_outputs)

        # Softmax normalization to obtain attention weights
        attention_weights = tf.nn.softmax(alignment_scores, axis=1)

        # Compute the context vector as the weighted sum of encoder outputs
        context_vector = tf.matmul(tf.expand_dims(attention_weights, axis=1), encoder_outputs)
        context_vector = tf.squeeze(context_vector, axis=1)

        return context_vector, attention_weights


# Example usage
if __name__ == "__main__":
    batch_size = 2
    seq_len = 5
    hidden_size = 10

    # Simulated inputs
    hidden = tf.random.normal([batch_size, hidden_size])  # Decoder hidden state
    encoder_outputs = tf.random.normal([batch_size, seq_len, hidden_size])  # Encoder outputs

    # Instantiate Luong Attention (dot, general, or concat)
    attention_type = "dot"  # Options: "dot", "general", "concat"
    attention_layer = LuongAttention(attention_type, hidden_size)

    # Forward pass
    context_vector, attention_weights = attention_layer(hidden, encoder_outputs)

    print("Context vector:", context_vector.numpy())
    print("Attention weights:", attention_weights.numpy())
